## Cell 1: Import thư viện và khai báo biến ban đầu

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

import os

data = []
labels = []
num_classes = 43
cur_path = os.getcwd()

## Cell 2: Đọc dữ liệu ảnh từ thư mục train và tạo tập dữ liệu

In [ ]:

for i in range(num_classes):
    path = os.path.join(cur_path, 'train', str(i))
    images = os.listdir(path)

    for a in images:
        try:
            image = Image.open(os.path.join(path, a))
            image = image.resize((30, 30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except Exception as e:
            print("Error loading image:", a, "in class", i, "-", e)

data = np.array(data)
labels = np.array(labels)

print("Data shape:", data.shape)
print("Labels shape:", labels.shape)

## Cell 3: Chia dữ liệu thành tập huấn luyện và kiểm tra (train/test split)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=43, stratify=labels
)

print("X_train:", X_train.shape)
print("X_test :", X_test.shape)
print("y_train:", y_train.shape)
print("y_test :", y_test.shape)


## Cell 4: Chuyển nhãn sang dạng one-hot encoding

In [ ]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print("y_train one-hot:", y_train.shape)
print("y_test one-hot :", y_test.shape)


## Cell 5: Xây dựng kiến trúc mạng CNN để nhận dạng biển báo

In [ ]:

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()


## Cell 6: Biên dịch (compile) và huấn luyện mô hình CNN

In [ ]:

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

epochs = 15
history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=epochs,
    validation_data=(X_test, y_test)
)


## Cell 7:Lưu mô hình đã huấn luyện và vẽ đồ thị Accuracy/Loss

In [ ]:

model.save("my_model.h5")

# Accuracy
plt.figure()
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss
plt.figure()
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
